In [ ]:
!pip install onnxruntime==1.9.0
!pip install -U tf2onnx==1.9.2

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import keras.layers
from sklearn.model_selection import train_test_split

from keras.models import Model
from keras.layers import Dense, Input
import math

import tf2onnx
import onnxruntime
from onnxruntime.quantization import quantize_dynamic


In [ ]:
OUTPUT_LABELS = ('fist_modifier', 'machete_modifier', 'arrow_modifier', 'grenade_modifier')

In [ ]:
def format_output(data):
  ys = list()
  for label in OUTPUT_LABELS:
    ys.append(np.array(data.pop(label)))
  return tuple(ys)


def norm(x, train_stats):
  return (x - train_stats['mean']) / train_stats['std']

In [ ]:
def laod_data(file_path):
  data = pd.read_csv(file_path)

  train, test = train_test_split(data, test_size=0.2, random_state = 1)
  train, val = train_test_split(train, test_size=0.2, random_state = 1)

  train_stats = train.describe()
  for label in OUTPUT_LABELS: train_stats.pop(label)
  train_stats = train_stats.transpose()

  train_Y = format_output(train)
  test_Y = format_output(test)
  val_Y = format_output(val)

  norm_train_X = np.array(norm(train, train_stats))
  norm_test_X = np.array(norm(test, train_stats))
  norm_val_X = np.array(norm(val, train_stats))

  x = norm_train_X, norm_test_X, norm_val_X
  y = train_Y, test_Y, val_Y

  return x, y, train

In [ ]:
def generate_model(model_path):
  x, y, train = laod_data('modifiers.csv')
  norm_train_X, norm_test_X, norm_val_X = x
  train_Y, test_Y, val_Y = y

  # Define model layers.
  input_layer = Input(shape=(len(train.columns),))
  first_dense = Dense(units=128, activation='relu')(input_layer)
  # Y1 output will be fed from the first dense
  y1_output = Dense(units=1, name='fist')(first_dense)

  second_dense = Dense(units=128, activation='relu')(first_dense)
  # Y2 output will be fed from the second dense
  y2_output = Dense(units=1, name='machete')(second_dense)

  third_dense = Dense(units=128, activation='relu')(first_dense)
  # Y3 output will be fed from the third dense
  y3_output = Dense(units=1, name='arrow')(third_dense)

  fourth_dense = Dense(units=128, activation='relu')(first_dense)
  # Y4 output will be fed from the fourth dense
  y4_output = Dense(units=1, name='grenade')(fourth_dense)

  # Define the model with the input layer and a list of output layers
  model = Model(inputs=input_layer, outputs=(y1_output, y2_output, y3_output, y4_output))

  model.compile(
              optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
              loss=dict(fist='mse', machete='mse', arrow='mse', grenade='mse'),
              metrics=dict(
                fist=tf.keras.metrics.RootMeanSquaredError(),
                machete=tf.keras.metrics.RootMeanSquaredError(),
                arrow=tf.keras.metrics.RootMeanSquaredError(),
                grenade=tf.keras.metrics.RootMeanSquaredError(),
              ),
  )

  history = model.fit(norm_train_X, train_Y,
                    epochs=200, batch_size=10, validation_data=(norm_test_X, test_Y))

  loss = model.evaluate(x=norm_val_X, y=val_Y)

  model.save(model_path, save_format='tf')

  return model


In [ ]:
def load_model(model_path):
  return keras.models.load_model(model_path)

In [ ]:
def variance(data, ddof=0):
  n = len(data)
  mean = sum(data) / n
  return sum((x - mean) ** 2 for x in data) / (n - ddof)


def stdev(data):
  var = variance(data)
  std_dev = math.sqrt(var)
  return std_dev

In [ ]:
model_path = 'damage_modifiers'
model = generate_model(model_path)

Epoch 1/200
72/72 [==============================] - 3s 13ms/step - loss: 4991.5464 - fist_loss: 1137.8550 - machete_loss: 1003.5477 - arrow_loss: 1420.1107 - grenade_loss: 1430.0337 - fist_root_mean_squared_error: 33.7321 - machete_root_mean_squared_error: 31.6788 - arrow_root_mean_squared_error: 37.6844 - grenade_root_mean_squared_error: 37.8158 - val_loss: 1082.9886 - val_fist_loss: 82.2840 - val_machete_loss: 148.9987 - val_arrow_loss: 613.4794 - val_grenade_loss: 238.2265 - val_fist_root_mean_squared_error: 9.0711 - val_machete_root_mean_squared_error: 12.2065 - val_arrow_root_mean_squared_error: 24.7685 - val_grenade_root_mean_squared_error: 15.4346
Epoch 2/200
72/72 [==============================] - 0s 3ms/step - loss: 2094.8628 - fist_loss: 160.1453 - machete_loss: 577.4286 - arrow_loss: 617.3281 - grenade_loss: 739.9608 - fist_root_mean_squared_error: 12.6549 - machete_root_mean_squared_error: 24.0297 - arrow_root_mean_squared_error: 24.8461 - grenade_root_mean_squared_error:

In [ ]:
model_path = 'damage_modifiers'
model = load_model(model_path)

In [ ]:
mean = lambda arr: sum(arr) / len(arr)

fist = [25, 10, 20, 30]
machete = [25, 20, 80, 30]
arrow = [25, 50, 50, 10]
grenade = [25, 30, 44, 30]

hits = [fist, machete, arrow, grenade]

params = list()
for h in hits:
  params.append((h[-1] - mean(h)) / stdev(h))

# print(params)

predictions = model.predict(np.array([params]))
print(predictions)

(array([[53.083138]], dtype=float32), array([[101.12877]], dtype=float32), array([[92.00295]], dtype=float32), array([[70.10308]], dtype=float32))


In [ ]:
onnx_model, _ = tf2onnx.convert.from_keras(model=model, opset=11)

In [ ]:
tf2onnx.onnx.save_model(onnx_model, model_path + '.onnx')

In [ ]:
sess = onnxruntime.InferenceSession(model_path + '.onnx')

In [ ]:
quantize_dynamic(model_path + '.onnx', model_path + '.q.onnx')